In [12]:
import json
import os
import mp
from glob import glob
from tqdm import tqdm
import soundfile as sf

timestamps = [i * 0.02 for i in range(1500 + 1)]

In [73]:
def chunk(alignment, reject = -8, minimum_length = 1.0):
    alls, temp = [], []
    for a in alignment:
        if a['score'] <= reject:
            if len(temp):
                temp[-1]['end'] = a['start']
                if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
                    alls.append(temp)
                temp = []
        else:
            temp.append(a)
            
    if len(temp):
        if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
            alls.append(temp)
    return alls

In [3]:
data = []
with open('prepared-pseudolabel-malaya.jsonl') as fopen:
    for no, l in tqdm(enumerate(fopen)):
        l = json.loads(l)
        data.append((no, l))
        
len(data)

1089630it [00:05, 202196.73it/s]


1089630

In [19]:
data[0][1]['new_text']

'<|startoftranscript|><|ms|><|transcribe|><|0.00|> kerajaan persekutuan<|1.46|><|1.46|> dan banyak masalah hubungan<|3.96|><|3.96|> antara kerajaan negeri dan<|5.70|><|endoftext|>'

In [74]:
!rm -rf prepared-pseudolabel-malaya-chunks
!mkdir prepared-pseudolabel-malaya-chunks

In [75]:
def loop(data):
    data, _ = data
    new_data = []
    for d in tqdm(data):
        no, l = d
        lang = l['new_text'].split('<|startoftranscript|><|')[1].split('|>')[0]
        f = f'prepared-pseudolabel-malaya_alignment/{no}.alignment'
        
        if not os.path.exists(f):
            continue
            
        with open(f) as fopen:
            align = json.load(fopen)
        
        chunks = chunk(align)
        if len(chunks):
            y, sr = sf.read(l['audio_filename'])
            for k, c in enumerate(chunks):
                skip = False
                
                for c_ in c:
                    if (c_['end'] - c_['start']) > 1:
                        skip = True
                        break
                if skip:
                    continue
                new_f = os.path.join('prepared-pseudolabel-malaya-chunks', f'{no}-{k}.mp3')
                if not os.path.exists(new_f):
                    sf.write(new_f, y[int(sr * c[0]['start']): int(sr * c[-1]['end'])], sr)
                ts = []
                
                min_t = min([c_['start'] for c_ in c])
                
                for c_ in c:
                    start = min(timestamps, key=lambda t: abs(t - (c_['start'] - min_t)))
                    end = min(timestamps, key=lambda t: abs(t - (c_['end'] - min_t)))
                    w = c_['text']
                    t = f"<|{start:.2f}|> {w}<|{end:.2f}|>"
                    ts.append(t)
                ts = ''.join(ts)
                new_text = text = f"<|startoftranscript|><|{lang}|><|transcribeprecise|>{ts}<|endoftext|>"
                new_data.append({
                    'audio_filename': new_f,
                    'new_text': new_text,
                })
                
    return new_data

In [76]:
data[0]

(0,
 {'new_text': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> kerajaan persekutuan<|1.46|><|1.46|> dan banyak masalah hubungan<|3.96|><|3.96|> antara kerajaan negeri dan<|5.70|><|endoftext|>',
  'audio_filename': 'output-audio-malaya/2-0-0.mp3'})

In [ ]:
r = loop((data[:10000], 0))

 19%|████████████████▎                                                                   | 1944/10000 [00:13<01:39, 80.84it/s]

In [84]:
len(r)

660

In [89]:
r[2]

{'audio_filename': 'prepared-pseudolabel-malaya-chunks/4-0.mp3',
 'new_text': '<|startoftranscript|><|ms|><|transcribeprecise|><|0.00|> kitab<|0.22|><|0.28|> dia,<|0.38|><|1.08|> maka<|1.26|><|1.32|> dia<|1.46|><|endoftext|>'}

In [90]:
import IPython.display as ipd
ipd.Audio(r[2]['audio_filename'])